## Feasibility for opening business in Frisco,Texas city.

### 1.Problem 
Evaluate business oppurtunity in Frisco,Texas area.

###### Problem Analysis
Identify Potential suburban/neighborrhood in and around Frisco, Analyize data about where poeple gathering most and identify the venues.         
         

### 2.Data Sources

Finding reports in Public domain to Identify the potential places  
http://money.com/money/5387468/best-places-to-live-2018-methodology/ 

http://money.com/money/collection/2018-best-places-to-live/ 

Based on public reports and  methodogy to identify best places Frisco is one of the best places to live. Due to this city selected to analyise its neighborhoods.

Available city Data from  Public domain

https://en.wikipedia.org/wiki/Frisco%2C_Texas

https://datausa.io/profile/geo/frisco-tx/#about

https://friscotexas.gov/

https://friscotexas.gov/176/GIS-Data-Download

https://www.friscotexas.gov/DocumentCenter/View/1446/HOA-Directory-PDF?bidId=

https://www.friscotexas.gov/1454/Population

http://www.friscohomesforsale.com/subdivisions-map.php

http://www.city-data.com/city/Frisco-Texas.html

https://www.usgs.gov/core-science-systems/ngp/board-on-geographic-names



In [2]:
# Install required libraries
#!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [3]:
# import libraries
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

###### Four square account setup

In [4]:
# The code was removed by Watson Studio for sharing.

### 3.Preprocess Data

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
# Get neigborhood info
body = client_60003201bf8e4290bfdf5cb593eef9c6.get_object(Bucket=bucketname,Key='Frisco_HOA_Directory.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_n = pd.read_csv(body)
df_data_n.head()


,Subdivision,Community
0,Northwest,CHRISTIE ESTATES
1,Northwest,CHRISTIE RANCH
2,Northwest,COBB FARM
3,Northwest,COBB HILL
4,Northwest,COUNTRY CLUB RIDGE AT THE TRAILS


##### Get  sub division co ordinates based on location andload data to dataframe 

In [7]:
# Function to get coordinates based on location and return with Dataframe
def getcoordinates(addrressList,colNames):
    data = []
    for address in addrressList:
        geolocator = Nominatim(user_agent="Frisco_explorer")
        location = geolocator.geocode(address)
        data.append([address,location.latitude,location.longitude])        
    df = pd.DataFrame(data,columns = colNames)
    return df

# 'Crestview,Frisco,TX' -Central,'Griffin Parc,Frisco,TX' -Northwest,'Stonebriar,Frisco,TX' -Southwest, 'Plantation Estates,Frisco,TX' -Southeast, 'Latera,Frisco,TX' -Northeast
address_list = ['Crestview,Frisco,TX','Griffin Parc,Frisco,TX','Stonebriar,Frisco,TX','Plantation Estates,Frisco,TX','Latera,Frisco,TX'] 
sub_division_List = ['Central','Northwest','Southwest','Southeast','Northeast']
cols = ['address','Latitude','Longitude']

df_sub = getcoordinates(address_list,cols) # get coordinate data
df_sub['Subdivision'] = sub_division_List # Add subdivision columnn 
del df_sub['address']
df_sub = df_sub[['Subdivision', 'Latitude', 'Longitude']]
df_sub.head()
   

,Subdivision,Latitude,Longitude
0,Central,33.160718,-96.805832
1,Northwest,33.175181,-96.861353
2,Southwest,33.099333,-96.811109
3,Southeast,33.118218,-96.844041
4,Northeast,33.160676,-96.757176


### 5.Store Data

In [8]:
# Merge data
Frisco_df = pd.merge(df_data_n, df_sub, on='Subdivision').reset_index(drop=True) # Combine
Frisco_df.head()

,Subdivision,Community,Latitude,Longitude
0,Northwest,CHRISTIE ESTATES,33.175181,-96.861353
1,Northwest,CHRISTIE RANCH,33.175181,-96.861353
2,Northwest,COBB FARM,33.175181,-96.861353
3,Northwest,COBB HILL,33.175181,-96.861353
4,Northwest,COUNTRY CLUB RIDGE AT THE TRAILS,33.175181,-96.861353


### 6.Analyize Data 

#####  Visualize Frisco city

In [9]:
#Get coordinates
address = 'Frisco, TX'
geolocator = Nominatim(user_agent="Frisco_Map")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# Frisco Map
map_Frisco = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(Frisco_df['Latitude'], Frisco_df['Longitude'], Frisco_df['Community']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Frisco)  
    
map_Frisco

##### Function to get  venues  information based on comunity coordinates 

In [10]:
def getFriscoVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Community', 
                  'Community Latitude', 
                  'Community Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Get Frisco venues

In [12]:
frisco_list = getFriscoVenues(names=Frisco_df['Community'],
                                   latitudes=Frisco_df['Latitude'],
                                   longitudes=Frisco_df['Longitude']
                                  )

CHRISTIE ESTATES
CHRISTIE RANCH
COBB FARM
COBB HILL
COUNTRY CLUB RIDGE AT THE TRAILS
ELDORADO FAIRWAYS AT THE TRAILS
EMERSON ESTATES
ESTATES AT ROCKHILL
GLEN VIEW
GRAYHAWK
GRIFFIN PARC
HEATHER RIDGE ESTATES
HOLLYHOCK
KINGS GARDEN
KNOLLS OF FRISCO
MEADOW CREEK
NEWMAN VILLAGE
NORTHRIDGE ESTATES
SHADDOCK CREEK ESTATES
SHADDOCK CREEK ESTATES
THE FRISCO FAIRWAYS
TRADITIONS OF FRISCO
TRAILS OF WEST FRISCO
VILLAS AT BELL CREEK
AVONDALE
BELMONT WOODS
BRETTON WOODS
CREEKSIDE AT PRESTON
CROWN RIDGE
CUSTER CREEK FARMS
DOMINION AT PANTHER CREEK
FRISCO HEIGHTS
HILLS OF CROWN RIDGE
INDEPENDENCE ESTATES
LATERA
LEXINGTON COUNTRY
MIRAMONTE
PANTHER CREEK ESTATES
PRAIRIE VIEW
RED BUD ESTATES
RIDGEVIEW AT PANTHER CREEK
ROCKHILL AT PRESTON
RUSSWOOD ACRES
SORANO
STONELAKE ESTATES
STONELAKE ESTATES WEST
VILLAGES AT WILLOW BAY
VILLAGES OF STONELAKE ESTATES
WINDING CREEK ESTATES
BELLA CASA
CANALS AT GRAND PARK
CHASE AT STONEBRIAR
CHEYENNE CROSSING
CREEKSIDE AT STONEBRIAR
DIAMOND POINT ESTATES
EDGESTONE AT LEGA

In [13]:
frisco_list.head()

,Community,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CHRISTIE ESTATES,33.175181,-96.861353,Sonic Drive In,33.175846,-96.865751,Burger Joint
1,CHRISTIE ESTATES,33.175181,-96.861353,Fairway Green Park,33.173816,-96.865235,Park
2,CHRISTIE ESTATES,33.175181,-96.861353,Friday Night,33.173827,-96.865254,Arts & Entertainment
3,CHRISTIE ESTATES,33.175181,-96.861353,Tulla Patisserie & Cafe,33.177480,-96.865155,Café
4,CHRISTIE ESTATES,33.175181,-96.861353,GPR,33.176596,-96.856509,Music Venue


######  Venues by comunity wise 

In [14]:
frisco_list.groupby('Community').count()


,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Community,,,,,,
AUTUMN PARK,14,14,14,14,14,14
AVONDALE,1,1,1,1,1,1
BELLA CASA,53,53,53,53,53,53
BELMONT WOODS,1,1,1,1,1,1
BRETTON WOODS,1,1,1,1,1,1
CANALS AT GRAND PARK,53,53,53,53,53,53
CECILE PLACE Ph1,14,14,14,14,14,14
CHAPEL CREEK,1,1,1,1,1,1
CHASE AT STONEBRIAR,53,53,53,53,53,53


##### Prepare data for analysis

In [18]:
# one hot encoding
frisco_onehot = pd.get_dummies(frisco_list[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
frisco_onehot['Community'] = frisco_list['Community'] 

# move neighborhood column to the first column
fixed_columns = [frisco_onehot.columns[-1]] + list(frisco_onehot.columns[:-1])
frisco_onehot = frisco_onehot[fixed_columns]

frisco_onehot.head()
frisco_onehot.shape


(3107, 55)

In [41]:
frisco_grouped = frisco_onehot.groupby('Community').mean().reset_index()
frisco_grouped

,Community,Accessories Store,American Restaurant,Arcade,Arts & Entertainment,Bank,Bookstore,Brazilian Restaurant,Burger Joint,Café,...,Southern / Soul Food Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Theme Park Ride / Attraction,Toy / Game Store,Video Game Store,Video Store,Yoga Studio
0,AUTUMN PARK,0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.0,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.071429
1,AVONDALE,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,BELLA CASA,0.037736,0.018868,0.018868,0.0,0.000000,0.018868,0.018868,0.0,0.000000,...,0.018868,0.018868,0.018868,0.018868,0.000000,0.037736,0.056604,0.018868,0.000000,0.000000
3,BELMONT WOODS,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,BRETTON WOODS,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,CANALS AT GRAND PARK,0.037736,0.018868,0.018868,0.0,0.000000,0.018868,0.018868,0.0,0.000000,...,0.018868,0.018868,0.018868,0.018868,0.000000,0.037736,0.056604,0.018868,0.000000,0.000000
6,CECILE PLACE Ph1,0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.0,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.071429
7,CHAPEL CREEK,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,CHASE AT STONEBRIAR,0.037736,0.018868,0.018868,0.0,0.000000,0.018868,0.018868,0.0,0.000000,...,0.018868,0.018868,0.018868,0.018868,0.000000,0.037736,0.056604,0.018868,0.000000,0.000000
9,CHEYENNE CROSSING,0.037736,0.018868,0.018868,0.0,0.000000,0.018868,0.018868,0.0,0.000000,...,0.018868,0.018868,0.018868,0.018868,0.000000,0.037736,0.056604,0.018868,0.000000,0.000000


##### Top venues comunity wise

In [44]:
num_top_venues = 5

for hood in frisco_grouped['Community']:
    print("----"+hood+"----")
    temp = frisco_grouped[frisco_grouped['Community'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AUTUMN PARK----
                  venue  freq
0  Fast Food Restaurant  0.14
1                  Bank  0.14
2                  Café  0.07
3    Salon / Barbershop  0.07
4    Mexican Restaurant  0.07


----AVONDALE----
                venue  freq
0                Park   1.0
1   Accessories Store   0.0
2          Restaurant   0.0
3  Mexican Restaurant   0.0
4       Movie Theater   0.0


----BELLA CASA----
               venue  freq
0     Clothing Store  0.08
1   Toy / Game Store  0.06
2  Accessories Store  0.04
3        Candy Store  0.04
4        Pizza Place  0.04


----BELMONT WOODS----
                venue  freq
0                Park   1.0
1   Accessories Store   0.0
2          Restaurant   0.0
3  Mexican Restaurant   0.0
4       Movie Theater   0.0


----BRETTON WOODS----
                venue  freq
0                Park   1.0
1   Accessories Store   0.0
2          Restaurant   0.0
3  Mexican Restaurant   0.0
4       Movie Theater   0.0


----CANALS AT GRAND PARK----
               

##### Add Top 5 venues to dataframe

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Community']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Community_venues_sorted = pd.DataFrame(columns=columns)
Community_venues_sorted['Community'] = frisco_grouped['Community']

for ind in np.arange(frisco_grouped.shape[0]):
    Community_venues_sorted.iloc[ind, 1:] = return_most_common_venues(frisco_grouped.iloc[ind, :], num_top_venues)

Community_venues_sorted.head()

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AUTUMN PARK,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
1,AVONDALE,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
2,BELLA CASA,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
3,BELMONT WOODS,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
4,BRETTON WOODS,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant


##### create Comunity clusters based on  sub division using K-Means cluster Approach

In [49]:
# set number of clusters
kclusters = 5

frisco_grouped_clustering = frisco_grouped.drop('Community', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(frisco_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 0, 2, 2, 0, 4, 1, 0, 0], dtype=int32)

In [54]:
# add clustering labels
Community_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

frisco_merged = Frisco_df

#merge frisco_grouped with toronto_data to add latitude/longitude for each neighborhood
frisco_merged = frisco_merged.join(Community_venues_sorted.set_index('Community'), on='Community')

frisco_merged.head() 

In [59]:
frisco_merged 

,Subdivision,Community,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Northwest,CHRISTIE ESTATES,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
1,Northwest,CHRISTIE RANCH,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
2,Northwest,COBB FARM,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
3,Northwest,COBB HILL,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
4,Northwest,COUNTRY CLUB RIDGE AT THE TRAILS,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
5,Northwest,ELDORADO FAIRWAYS AT THE TRAILS,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
6,Northwest,EMERSON ESTATES,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
7,Northwest,ESTATES AT ROCKHILL,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
8,Northwest,GLEN VIEW,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
9,Northwest,GRAYHAWK,33.175181,-96.861353,3,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant


##### Visualize  sub division clusters

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(frisco_merged['Latitude'], frisco_merged['Longitude'], frisco_merged['Community'], frisco_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 7. Evaluate Results

##### 'SouthWest' sub division cluster data

In [65]:
frisco_merged.loc[frisco_merged['Cluster Labels'] == 0, frisco_merged.columns[[1] + list(range(5, frisco_merged.shape[1]))]]

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,BELLA CASA,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
50,CANALS AT GRAND PARK,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
51,CHASE AT STONEBRIAR,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
52,CHEYENNE CROSSING,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
53,CREEKSIDE AT STONEBRIAR,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
54,DIAMOND POINT ESTATES,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
55,EDGESTONE AT LEGACY,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
56,ENCLAVE AT STONEBRIAR CREEK ESTATES,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
57,ESTATES ON LEGACY,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store
58,FAIRWATER,Clothing Store,Toy / Game Store,Lingerie Store,Shoe Store,Candy Store,Cosmetics Shop,Department Store,Furniture / Home Store,Pizza Place,Accessories Store


#####  'Central' sub division cluster  data

In [66]:
frisco_merged.loc[frisco_merged['Cluster Labels'] == 1, frisco_merged.columns[[1] + list(range(5, frisco_merged.shape[1]))]]

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
136,CHAPEL CREEK,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
137,CHRISTIE AND WESTER,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
138,COLLINSWORTH,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
139,COVENTRY OAKS,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
140,CRESTVIEW,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
141,EVA DELL,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
142,FRISCO SQUARE TOWNHOMES,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
143,GOLDEN GATE VILLAGE,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
144,GRIFFIN,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store
145,HICKORY HILLS,Café,Yoga Studio,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store


#####  'NorthEast' sub division  cluster data

In [67]:
frisco_merged.loc[frisco_merged['Cluster Labels'] == 2, frisco_merged.columns[[1] + list(range(5, frisco_merged.shape[1]))]]

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,AVONDALE,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
25,BELMONT WOODS,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
26,BRETTON WOODS,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
27,CREEKSIDE AT PRESTON,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
28,CROWN RIDGE,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
29,CUSTER CREEK FARMS,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
30,DOMINION AT PANTHER CREEK,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
31,FRISCO HEIGHTS,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
32,HILLS OF CROWN RIDGE,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant
33,INDEPENDENCE ESTATES,Park,Yoga Studio,Cosmetics Shop,Italian Restaurant,Intersection,Indian Restaurant,Indian Chinese Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant


#####  'Northwest'  subdivision cluster  data

In [68]:
frisco_merged.loc[frisco_merged['Cluster Labels'] == 3, frisco_merged.columns[[1] + list(range(5, frisco_merged.shape[1]))]]

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CHRISTIE ESTATES,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
1,CHRISTIE RANCH,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
2,COBB FARM,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
3,COBB HILL,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
4,COUNTRY CLUB RIDGE AT THE TRAILS,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
5,ELDORADO FAIRWAYS AT THE TRAILS,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
6,EMERSON ESTATES,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
7,ESTATES AT ROCKHILL,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
8,GLEN VIEW,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant
9,GRAYHAWK,Arts & Entertainment,Music Venue,Burger Joint,Café,Park,Yoga Studio,Department Store,Italian Restaurant,Intersection,Indian Restaurant


#####  'SouthEast' Sub division  cluster data

In [69]:
frisco_merged.loc[frisco_merged['Cluster Labels'] == 4, frisco_merged.columns[[1] + list(range(5, frisco_merged.shape[1]))]]

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,AUTUMN PARK,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
94,CECILE PLACE Ph1,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
95,CREEKS ON COIT,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
96,CYPRESS CREEK,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
97,EMORY PARK,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
98,FAIRFIELD ESTATES,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
99,HILLCREST ESTATES,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
100,HILLCREST ESTATES,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
101,HILLCREST HIGHLANDS,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy
102,HILLCREST LEBANON,Fast Food Restaurant,Bank,Yoga Studio,Intersection,Optical Shop,Nail Salon,Salon / Barbershop,Café,Mexican Restaurant,Pharmacy


### 8. Conclusion

Frisco subdivisions exists with venues like cafe, shopping, Reastaurants, Bank in each sub divisios but not exists with any sports business domain.So there is good oppurtunity to open Sports bar between southeast,central,southwest sub-divisions  since this sbdivisions sourrended with sports facilitie like Toyota stadium, Dallas cowboys training facility, Ice skating arena.

*Based on data availble in public domain 